<a href="https://colab.research.google.com/github/Prajwalng2/Credit-card-fraud-detection/blob/main/credit_card_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv('creditcard.csv')

In [2]:
df.shape
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [3]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
df[['Amount', 'Time']] = scaler.fit_transform(df[['Amount', 'Time']])


In [5]:
df = df.sample(frac=1, random_state=42)
df

not_frauds = df[df['Class'] == 0]
frauds = df[df['Class'] == 1]

balanced_df = pd.concat([frauds, not_frauds.sample(n=len(frauds), random_state=42)])

balanced_df = balanced_df.sample(frac=1, random_state=42)

balanced_df['Class'].value_counts()


Class
1.0    3
0.0    3
Name: count, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
balanced_df_np = balanced_df.to_numpy()


x_train_val_b, x_test_b, y_train_val_b, y_test_b = train_test_split(
    balanced_df_np[:, :-1], balanced_df_np[:, -1].astype(int), test_size=0.15, random_state=42
)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(
    x_train_val_b, y_train_val_b, test_size=0.15, random_state=42
)


x_train_b.shape, y_train_b.shape, x_test_b.shape, y_test_b.shape, x_val_b.shape, y_val_b.shape



pd.Series(y_train_b).value_counts(), pd.Series(y_test_b).value_counts(), pd.Series(y_val_b).value_counts()


(1    2
 0    2
 Name: count, dtype: int64,
 1    1
 Name: count, dtype: int64,
 0    1
 Name: count, dtype: int64)

In [9]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense, BatchNormalization

shallow_nn = Sequential()
shallow_nn.add(InputLayer(input_shape=(x_train_b.shape[1],)))
shallow_nn.add(Dense(2, activation='relu'))
shallow_nn.add(BatchNormalization())
shallow_nn.add(Dense(1, activation='sigmoid'))


In [11]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('shallow_nn', save_best_only=True)

shallow_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

shallow_nn.fit(x_train_b, y_train_b, validation_data=(x_val_b, y_val_b), epochs=40, callbacks=[checkpoint])


Epoch 1/40
1/1 [==============================] - 2s 2s/step - loss: 0.3355 - accuracy: 0.7500 - val_loss: 1.5481 - val_accuracy: 0.0000e+00
Epoch 2/40
1/1 [==============================] - 1s 863ms/step - loss: 0.3215 - accuracy: 0.7500 - val_loss: 1.5370 - val_accuracy: 0.0000e+00
Epoch 3/40
1/1 [==============================] - 1s 918ms/step - loss: 0.3079 - accuracy: 0.7500 - val_loss: 1.5254 - val_accuracy: 0.0000e+00
Epoch 4/40
1/1 [==============================] - 1s 956ms/step - loss: 0.2946 - accuracy: 0.7500 - val_loss: 1.5135 - val_accuracy: 0.0000e+00
Epoch 5/40
1/1 [==============================] - 1s 670ms/step - loss: 0.2816 - accuracy: 0.7500 - val_loss: 1.5015 - val_accuracy: 0.0000e+00
Epoch 6/40
1/1 [==============================] - 0s 475ms/step - loss: 0.2690 - accuracy: 0.7500 - val_loss: 1.4894 - val_accuracy: 0.0000e+00
Epoch 7/40
1/1 [==============================] - 0s 474ms/step - loss: 0.2569 - accuracy: 1.0000 - val_loss: 1.4773 - val_accuracy: 0.0000

In [14]:

shallow_nn_b = shallow_nn


shallow_nn_b.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


shallow_nn_b.fit(x_train_b, y_train_b, validation_data=(x_val_b, y_val_b), epochs=40)

neural_net_predictions(shallow_nn_b, x_val_b)


Epoch 1/40
1/1 [==============================] - 1s 725ms/step - loss: 0.0775 - accuracy: 1.0000 - val_loss: 0.5128 - val_accuracy: 1.0000
Epoch 2/40
1/1 [==============================] - 0s 26ms/step - loss: 0.0771 - accuracy: 1.0000 - val_loss: 0.5022 - val_accuracy: 1.0000
Epoch 3/40
1/1 [==============================] - 0s 24ms/step - loss: 0.0767 - accuracy: 1.0000 - val_loss: 0.4921 - val_accuracy: 1.0000
Epoch 4/40
1/1 [==============================] - 0s 24ms/step - loss: 0.0762 - accuracy: 1.0000 - val_loss: 0.4823 - val_accuracy: 1.0000
Epoch 5/40
1/1 [==============================] - 0s 24ms/step - loss: 0.0758 - accuracy: 1.0000 - val_loss: 0.4721 - val_accuracy: 1.0000
Epoch 6/40
1/1 [==============================] - 0s 24ms/step - loss: 0.0754 - accuracy: 1.0000 - val_loss: 0.4615 - val_accuracy: 1.0000
Epoch 7/40
1/1 [==============================] - 0s 34ms/step - loss: 0.0750 - accuracy: 1.0000 - val_loss: 0.4507 - val_accuracy: 1.0000
Epoch 8/40
1/1 [==========

array([0])

In [16]:
from sklearn.metrics import classification_report


predictions = neural_net_predictions(shallow_nn, x_val_b)


print(classification_report(y_val_b, predictions, labels=[0, 1], target_names=['Not Fraud', 'Fraud']))


1/1 [==============================] - 0s 15ms/step
              precision    recall  f1-score   support

   Not Fraud       1.00      1.00      1.00         1
       Fraud       0.00      0.00      0.00         0

   micro avg       1.00      1.00      1.00         1
   macro avg       0.50      0.50      0.50         1
weighted avg       1.00      1.00      1.00         1



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [17]:
shallow_nn.save("shallow_nn_b_model.n5")